# Drive 접근해서 Dataset 로드

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 드라이브 내 경로로 이동
%cd /content/drive/MyDrive/oss_drive

/content/drive/MyDrive/oss_drive


In [ ]:
'''# 이미지 데이터셋 (알약 22개 품목) 압축해제해서 'dataset'폴더에 저장
!unzip /content/drive/MyDrive/oss_drive/sample_pills.zip -d /content/drive/MyDrive/oss_drive/original'''

# Preprocessing
## 1. Crop & Rotating
original 폴더의 원본사진을 크롭하여 cropped 폴더에 크롭사진을 저장하고 (1:1), <br>
cropped 폴더의 크롭사진을 5도씩 회전시켜 rotated 폴더에 저장한다. (1:73)

In [ ]:
import cv2
import numpy as np
import os

class ImageCrop():
    def __init__(self):
        '''
        open_path : original image open path
        save_path : crop image save path
        rotate_path : rotate image save path
        rotation_angle_circle : circle rotation angle
        rotation_angle_ellipse : ellipse rotation angle
        '''

        self.open_path = '/content/drive/MyDrive/oss_drive/original/'
        self.save_path = '/content/drive/MyDrive/oss_drive/cropped/'
        self.rotate_path = '/content/drive/MyDrive/oss_drive/rotated/'
        self.rotate_angle_circle = int(5) # 5도씩 회전
        self.rotate_angle_ellipse = int(5) # 5도씩 회전

        # Permission Error retry another path
        self.error_path_crop = './crop'
        self.error_path_rotation = './rotation'

        # dir index setting
        self.start_dir = -1
        self.end_dir = -1


    # square four point return
    def ImageArea(self, input_image):
        rgba = cv2.medianBlur(input_image, 55)

        imgray = cv2.cvtColor(rgba, cv2.COLOR_BGRA2GRAY)
        contours, _ = cv2.findContours(imgray, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        contours_t = contours[0].transpose()

        right_point_x = np.max(contours_t[0])
        left_point_x = np.min(contours_t[0])
        right_point_y = np.max(contours_t[1])
        left_point_y = np.min(contours_t[1])

        return left_point_x, right_point_x, left_point_y, right_point_y


    # image crop
    def CropShape(self, input_image):
        left_x, right_x, left_y, right_y = self.ImageArea(input_image)
        crop_img = input_image[left_y:right_y, left_x:right_x]

        return crop_img


    # circle image rotation - 원 회전
    def rotate_image_circle(self, save_rotate_img, input_image):
        i = 0
        height, width, channel = input_image.shape

        while i < 360:
            f_path = save_rotate_img + '_' + str(i) + '.png'
            if not os.path.isfile(f_path):
                matrix = cv2.getRotationMatrix2D((width/2, height/2), i, 1)
                dst = cv2.warpAffine(input_image, matrix, (width, height))
                dst = self.CropShape(dst)

                cv2.imwrite(f_path, dst)
            else:
                print('rotate file exits : ', f_path)

            i = i + self.rotate_angle_circle


    # ellipse image rotation - 타원 회전
    def rotate_image_ellipse(self, save_rotate_img, input_image):
        i = 0
        height, width, channel = input_image.shape

        while i < 360:
            if (i < 30) or (150 < i and i < 210) or (330 < i):
                f_path = save_rotate_img + '_' + str(i) + '.png'
                if not os.path.isfile(f_path):
                    matrix = cv2.getRotationMatrix2D((width/2, height/2), i, 1)
                    dst = cv2.warpAffine(input_image, matrix, (width, height))
                    dst = self.CropShape(dst)

                    cv2.imwrite(f_path, dst)
                else:
                    print('rotate file exits : ', f_path)

            i = i + self.rotate_angle_ellipse


    # image crop and rotation process - 전체 과정
    def ImageProcess(self, shape):
        or_dirnames = os.listdir(self.open_path) # 원본디렉토리 내 품목디렉토리들

        if( int(self.start_dir) == -1 ):
            dirnames = or_dirnames
        else:
            dirnames = or_dirnames[int(self.start_dir):int(self.end_dir)]

        for dir in dirnames: # 각 품목디렉토리에 대하여
            try_num = 0
            # try
            try:
                # not exists folder in path, make folder
                if not os.path.exists(self.save_path + dir):
                    os.makedirs(self.save_path + '/' + dir)

                if not os.path.exists(self.rotate_path + dir):
                    os.makedirs(self.rotate_path + '/' + dir)
                try_num = 1

            except PermissionError:
                if not os.path.exists(self.error_path_crop + dir):
                    os.makedirs(self.error_path_crop + '/' + dir)
                    print("PermissionError except, image save path: ", self.error_path_crop)

                if not os.path.exists(self.error_path_rotation + dir):
                    os.makedirs(self.error_path_rotation + '/' + dir)
                    print("PermissionError except, image save path: ", self.error_path_rotation)
                try_num = 2

            except IOError as e:
                print("IOError except: ", e.errno)

            open_folder_path = os.path.join(self.open_path, dir)
            if try_num == 1:
                save_folder_path = os.path.join(self.save_path, dir)
                rotate_folder_path = os.path.join(self.rotate_path, dir)
            elif try_num == 2:
                save_folder_path = os.path.join(self.error_path_crop, dir)
                rotate_folder_path = os.path.join(self.error_path_rotation, dir)

            for path, folder, files in os.walk(open_folder_path):

                for file in files:
                    input_image = open_folder_path + '/' + file

                    print(input_image)

                    save_image = save_folder_path + '/' + file[0:len(file)-3] + 'png'
                    input_image = cv2.imread(input_image, cv2.IMREAD_UNCHANGED)

                    '''image crop'''
                    if not os.path.isfile(save_image):
                        crop_img = self.CropShape(input_image)
                        cv2.imwrite(save_image, crop_img)
                    else:
                        print( 'crop image file exits : ', save_image)

                    '''rotation''' # shape에 따라 다른 회전 필요
                    save_rotate_img = rotate_folder_path + '/' + file[0:len(file)-4]

                    if shape == 'circle':
                        self.rotate_image_circle(save_rotate_img, input_image)
                    elif shape == 'ellipse':
                        self.rotate_image_ellipse(save_rotate_img, input_image)


In [ ]:
# crop & rotating 테스트 --> 테스트 결과 bb
# ex) python3 PreProcessing.py circle config.ini (나중에는 config.ini로 파라미터 값 설정할 것)
shape = 'circle'
ImgCrop = ImageCrop()
ImgCrop.ImageProcess(shape)
print('####### finish #######')

/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-06-671.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-17-686.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-49-106.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-07-673.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-59-121.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-06-03-125.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-48-106.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-53-111.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-01-663.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-50-107.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-05-671.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-05-13-683.png
/content/drive/MyDrive/oss_drive/original/40122/2020-10-29-21-06-00-121.png
/content/dri

## 2. Filtering
rotated 폴더의 회전사진으로부터 filtering을 통해 배경 white로 변경하여 filetered 폴더에 필터링사진 저장한다. (1:1)

In [6]:
import cv2
import os
import errno

class ImageFiltering():
    def __init__(self):
        '''
        rotate_path : rotate image save path
        filter_path : filter image save path
        '''

        self.origin_folder_path = '/content/drive/MyDrive/oss_drive/rotated'
        self.filter_folder_path = '/content/drive/MyDrive/oss_drive/filtered'

        # Permission Error retry another path
        self.error_path_filter = './filter'


    # make white background - image 배경을 white로 바꿔서 저장
    def white_Background(self, img):
        trans_mask = img[:,:,3]==0
        img[trans_mask] = [255, 255, 255, 255]
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        return img


    # filter (컬러모델의 l채널 변경하여 적용)
    def max_con_CLAHE(self, img):
        # Converting image to LAB Color model
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)

        # Applying CLAHE to L-channel
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)

        # Merge the CLAHE enhanced L-channel with the a and b channel
        limg = cv2.merge((cl,a,b))

        # Converting image from LAB Color model to RGB model
        img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

        return img

    # if image open fail, write log of fail image full path
    def tracelog(self, text):
        if text != "":
            text += "\n"
            f = open("tracelog.log","a")
            f.write(text)
            f.close()


    # filtering processing (실제 실행 과정)
    def imgFiltering(self):
        dirnames = os.listdir(self.origin_folder_path)

        for filename in dirnames: # 디렉토리당 파일 각각에 대하여
            try_num = 0
            try:
                if not os.path.exists(self.filter_folder_path + filename):
                    os.makedirs(self.filter_folder_path + filename)
                try_num = 1

            except PermissionError:
                if not os.path.exists(self.error_path_filter + filename):
                    os.makedirs(self.error_path_filter + filename)
                    print("PermissionError except, image save path: ", self.error_path_filter)
                try_num = 2

            except IOError as e:
                print("IOError except: ", e.errno)

            origin_file_path = os.path.join(self.origin_folder_path, filename)
            if try_num == 1:
                filter_file_path = os.path.join(self.filter_folder_path, filename)
            elif try_num == 2:
                filter_file_path = os.path.join(self.error_path_filter, filename)

            for path, folder, files in os.walk(origin_file_path):

                for file in files: # 디렉토리 내 파일들 중 각 파일에 대하여
                    print(filter_file_path+'/' + file)
                    print("origin path : ", origin_file_path +'/'+ file)

                    # file check
                    save_path = filter_file_path + '/' + file[0:len(file)-4] + '.jpg'
                    if os.path.isfile(save_path):
                        print("filter file exists : {}".format(save_path))
                        continue

                    img = cv2.imread(origin_file_path +'/'+ file, cv2.IMREAD_UNCHANGED)

                    # image check
                    if img is None:
                        self.tracelog(origin_file_path +'/'+ file)
                        print('typecheck ', type(img))
                        continue

                    ''' white background '''
                    img = self.white_Background(img)

                    ''' default filter '''
                    img = self.max_con_CLAHE(img)
                    img = self.max_con_CLAHE(img)

                    ''' filtering image save '''
                    cv2.imwrite(save_path , img)


In [7]:
# filtering 테스트 --> 7m 25s. 잘못된 결과 (에러경로, 빈폴더)
# ex) python3 PreProcessing.py circle config.ini (나중에는 config.ini로 파라미터 값 설정할 것)
ImageFiltering = ImageFiltering()
ImageFiltering.imgFiltering()
print('####### finish #######')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
origin path :  /content/drive/MyDrive/oss_drive/rotated/40720/IMG_20201202_001106_25.png
/content/drive/MyDrive/oss_drive/filtered/40720/IMG_20201202_001106_30.png
origin path :  /content/drive/MyDrive/oss_drive/rotated/40720/IMG_20201202_001106_30.png
/content/drive/MyDrive/oss_drive/filtered/40720/IMG_20201202_001106_35.png
origin path :  /content/drive/MyDrive/oss_drive/rotated/40720/IMG_20201202_001106_35.png
/content/drive/MyDrive/oss_drive/filtered/40720/IMG_20201202_001106_40.png
origin path :  /content/drive/MyDrive/oss_drive/rotated/40720/IMG_20201202_001106_40.png
/content/drive/MyDrive/oss_drive/filtered/40720/IMG_20201202_001106_45.png
origin path :  /content/drive/MyDrive/oss_drive/rotated/40720/IMG_20201202_001106_45.png
/content/drive/MyDrive/oss_drive/filtered/40720/IMG_20201202_001106_50.png
origin path :  /content/drive/MyDrive/oss_drive/rotated/40720/IMG_20201202_001106_50.png
/content/drive/MyDrive/oss_drive/filtered/40720/IMG_202

## 3. Seperating
폴더 분류하는 task 수행. PIL(pillow)내 Image모듈 사용함

In [ ]:
import os
import shutil
from PIL import Image


class Separate():
    def __init__(self, config):
        '''
        filter_folder_path : filter image save path
        folder_save_path : traing/validation/traing separate image folder save path
        '''

        self.open_path = '/content/drive/MyDrive/oss_drive/rotated'
        self.save_path = '/content/drive/MyDrive/oss_drive/seperated'


    def FolderList(self):
        type_list = []
        folder_list = []

        for (path, folder, files) in os.walk(self.open_path):
            type_list.append(path)
            folder_list.append(folder)

        folders = ','.join(folder_list[0])
        folder_list = folders.split(',')
        type_list.pop(0)

        return folder_list


    def makeSubfolder(self, new_dir_path, folder_list):
        for num in range(len(folder_list)):
            os.makedirs(new_dir_path + folder_list[num])


    def ml_directory(self, folder_list):
        if not os.path.exists(self.save_path + 'training'):
            os.makedirs(self.save_path + 'training')
            self.makeSubfolder(self.save_path + 'training/', folder_list)

        if not os.path.exists(self.save_path + 'testing'):
            os.makedirs(self.save_path + 'testing')
            self.makeSubfolder(self.save_path + 'testing/', folder_list)

        if not os.path.exists(self.save_path + 'validation'):
            os.makedirs(self.save_path + 'validation')
            self.makeSubfolder(self.save_path + 'validation/', folder_list)


    def separate(self, dir_path, x):
        dirname = self.open_path + x
        filenames = os.listdir(dirname)
        i = 0
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)

            with Image.open(full_filename) as image:
                if i % 10 < 7:
                    training_directory = os.path.join(dir_path + 'training/', x)
                    shutil.copyfile(full_filename, os.path.join(training_directory, filename))

                elif i % 10 >= 7 and i % 10 < 8:
                    validation_directory = os.path.join(dir_path + 'testing/', x)
                    shutil.copyfile(full_filename, os.path.join(validation_directory, filename))

                else:
                    testing_directory = os.path.join(dir_path + 'validation/', x)
                    shutil.copyfile(full_filename, os.path.join(testing_directory, filename))
            i = i + 1


    def separateProcess(self):
        folder_list = self.FolderList()
        self.ml_directory(folder_list)

        for x in folder_list:
            self.separate(self.save_path, x)



## Main
전체 태스크 수행. 추후에 태스크별로 파일 분류하는 거 고려해봐야 할 듯

In [ ]:
'''import ImageCrop as ImageCrop
import ImageFiltering as ImageFiltering
import Separate as Separate'''
import configparser
import sys


class PreprocessingMain():
    def __init__(self):
        self.config_file = '/content/drive/MyDrive/oss_drive/preprocessing_config.ini'

    def main(self, argv):
        # ex) python3 PreProcessing.py circle config.ini
        if len(argv) == 3:
            self.config_file = argv[2]

        shape = argv[1]

        config = configparser.ConfigParser()
        config.read(self.config_file, encoding='UTF-8')

        self.ImgCrop = ImageCrop.ImageCrop(config['img_processing'])
        self.ImgFilter = ImageFiltering.ImageFiltering(config['img_processing'])
        self.Separate = Separate.Separate(config['img_processing'])


        """ Image Crop """
        self.ImgCrop.ImageProcess(shape)

        """ Image Filtering """
        self.ImgFilter.imgFiltering()

        """ Image Separte"""
        self.Separate.separateProcess()

        print('####### finish #######')


        if __name__ == '__main__':
            main_class = PreprocessingMain()
            main_class.main(sys.argv)


In [ ]:
!oss_data.ipynb

# Training

## 파이토치 버전 확인

In [ ]:
'''# colab에서 기본제공하는 내장패키지 확인
!pip3 list'''

'# colab에서 기본제공하는 내장패키지 확인\n!pip3 list'

In [ ]:
'''
# 파이토치 버전 확인
import torch
import torchvision
#import torchtext

print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')
#print(f'torchtext version: {torchtext.__version__}')
'''

"\n# 파이토치 버전 확인\nimport torch\nimport torchvision\n#import torchtext\n\nprint(f'torch version: {torch.__version__}')\nprint(f'torchvision version: {torchvision.__version__}')\n#print(f'torchtext version: {torchtext.__version__}')\n"

In [ ]:
'''
# 문제가 있다면, 설치
!pip3 install torch
!pip3 install torchvision
'''

'\n# 문제가 있다면, 설치\n!pip3 install torch\n!pip3 install torchvision\n'

## 모델 정의

In [ ]:
from torch import nn

# 파이토치 함수 정의
class SuperLightMobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SuperLightMobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )
        self.num_classes = num_classes
        self.model = nn.Sequential(
            conv_bn(  3,  16, 2),
            conv_dw( 16,  32, 1),
            conv_dw( 32, 64, 2),
            conv_dw(64, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.gap(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

#  Hyper Paramertes & Device Setting

In [ ]:
import torch

#hyp parameters
dataset_path = "/content/drive/MyDrive/oss_drive/dataset/"
model_weight_save_path = "/content/drive/MyDrive/oss_drvie/model/"
num_classes = 2

batch_size = 16
num_workers = 8
lr = 1e-3

total_epoch = 30
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Dataset & Downloader Setting

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import os

# Data loading code
traindir = os.path.join(dataset_path, 'train')
testdir = os.path.join(dataset_path, 'test')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)